## Prepare dataset for fine-tuning with mp_20

In [2]:
import pandas as pd
df = pd.read_csv("../data/mp_20/mp_20.csv")
df.head(2)

,Unnamed: 0,material_id,formation_energy_per_atom,band_gap,pretty_formula,e_above_hull,elements,cif,spacegroup.number
0,37228,mp-1221227,-1.637460,0.2133,Na3MnCoNiO6,0.043001,"['Co', 'Mn', 'Na', 'Ni', 'O']",# generated using pymatgen\r\ndata_Na3MnCoNiO6...,8
1,19480,mp-974729,-0.314759,0.0000,Nd(Al2Cu)4,0.000000,"['Al', 'Cu', 'Nd']",# generated using pymatgen\r\ndata_Nd(Al2Cu)4\...,139


In [6]:
def create_input(row):
    return f"band_gap={row['band_gap']} spacegroup.number={row['spacegroup.number']}"

df['cif'] = df['cif'].str.replace('# generated using pymatgen\n', '', regex=False)

new_data = {
    'input': df.apply(create_input, axis=1),
    'output': df['cif']
}

df_for_llama = pd.DataFrame(new_data)
df_for_llama.to_csv('../data/df_for_llama.csv', index=False)

In [7]:
df_for_llama.head(2)

,input,output
0,band_gap=0.2132999999999998 spacegroup.number=8,data_Na3MnCoNiO6\n_symmetry_space_group_name_H...
1,band_gap=0.0 spacegroup.number=139,data_Nd(Al2Cu)4\n_symmetry_space_group_name_H-...


## Create SLICE dataset for fine-tune

In [4]:
import pandas as pd
df = pd.read_csv("../data/alexandria/alex_full_slice.csv")
df

,Unnamed: 0,mat_id,prototype_id,location,formula,elements,spg,nsites,stress,energy_total,...,dos_ef,energy_corrected,e_above_hull,e_form,e_phase_separation,decomposition,in_nomad,cif,SLICE,SLICE PLUS
0,61866,agm004511222,AB2C3D4_7_spg115,cgat_comp/quaternaries/AB2C3D4/runs/batch-000/...,Sr2Zn3Ge4Pd,"['Sr', 'Zn', 'Ge', 'Pd']",115,10,"[[3.0763032, 0.0, 0.0], [0.0, 3.0763032, 0.0],...",-35.283144,...,2.885905,-35.283146,0.025135,-0.440718,0.021063,Sr2Zn3Ge4Pd,True,# generated using pymatgen\ndata_Sr2Zn3Ge4Pd\n...,Sr Sr Zn Zn Zn Ge Ge Ge Ge Pd 0 7 o o o 1 8 o ...,Sr Sr Zn Zn Zn Ge Ge Ge Ge Pd 0 7 o o o 1 8 o ...
1,61867,agm003932735,ABC2_7003_spg71,extra/batch-3000/Cl/Cl2CrTc/xxx_02a-00_agm0039...,CrTcCl2,"['Cr', 'Tc', 'Cl']",12,4,"[[2.9210932, 0.0116612, 0.05841717], [0.011661...",-21.772225,...,3.483516,-21.772224,1.225197,0.181212,1.225197,CrCl3 Cr Tc,True,# generated using pymatgen\ndata_CrTcCl2\n_sym...,Cr Tc Cl Cl 0 2 + + o 0 1 o o o 1 2 o + o 1 3 ...,Cr Tc Cl Cl 0 2 + + o 0 1 o o o 1 2 o + o 1 3 ...
2,61868,agm003160492,AB2_0_spg139,extra/batch-003/Au/AuTl2/xxx_02a-00_agm0031604...,Tl2Au,"['Tl', 'Au']",139,3,"[[4.422527, -0.01632697, -3.91e-06], [-0.01632...",-8.074673,...,1.489964,-8.074674,0.026976,-0.021588,0.003966,Tl2Au,NaN,# generated using pymatgen\ndata_Tl2Au\n_symme...,Tl Tl Au 0 1 + o o 0 1 o o o 0 2 + o + 1 2 o o +,Tl Tl Au 0 1 + o o 0 1 o o o 0 2 + o + 1 2 o o...
3,61869,agm002371989,ABC2_18_spg123,substitutions/batch-005/Fe/FePt2Pu/xxx_02p-00_...,PuFePt2,"['Fe', 'Pt', 'Pu']",123,4,"[[4.6794424, 0.0, 0.0], [0.0, 4.7021456, 0.0],...",-34.758170,...,14.017008,-34.758170,0.669855,0.029511,0.669855,PuPt2 Fe,True,# generated using pymatgen\ndata_PuFePt2\n_sym...,Pu Fe Pt Pt 0 1 o - o 2 3 o o -,Pu Fe Pt Pt 0 1 o - o 2 3 o o - i u y 1s 1s 1s...
4,61870,agm001640577,ABCD2_2_spg123,cgat_prot/ABCD2_prot2_spg123/runs/batch-102/Zn...,SrZn2CdPd,"['Sr', 'Zn', 'Pd', 'Cd']",123,5,"[[3.8065994, 0.0, 0.0], [0.0, 3.8065994, 0.0],...",-11.628009,...,2.942810,-11.628009,0.182077,-0.253307,0.182077,SrZn9Pd2 SrCd2 SrZn3Pd2,True,# generated using pymatgen\ndata_SrZn2CdPd\n_s...,Sr Zn Zn Cd Pd 0 4 o o + 0 3 o o o 0 3 o + o 0...,Sr Zn Zn Cd Pd 0 4 o o + 0 3 o o o 0 3 o + o 0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5068739,61861,agm001651623,ABCD2_2_spg123,cgat_prot/ABCD2_prot2_spg123/runs/batch-103/Ca...,Sr2CaZnPd,"['Sr', 'Zn', 'Ca', 'Pd']",123,5,"[[1.4767605, 0.0, 0.0], [0.0, 1.4767605, 0.0],...",-12.761735,...,3.073766,-12.761735,0.217014,-0.182018,0.217014,Sr13Pd2 Ca2ZnPd Sr2ZnPd SrZn,True,# generated using pymatgen\ndata_Sr2CaZnPd\n_s...,Sr Sr Ca Zn Pd 0 4 + o + 1 4 + + o 2 3 o - -,Sr Sr Ca Zn Pd 0 4 + o + 1 4 + + o 2 3 o - - i...
5068740,61862,agm001403142,ABCD_1_spg216,cgat_prot/ABCD_prot1_spg216/runs/batch-001/Cr/...,MnCrCoRe,"['Re', 'Mn', 'Co', 'Cr']",216,4,"[[5.4412737, 0.0, 0.0], [0.0, 5.4412737, 0.0],...",-38.182897,...,2.123811,-38.182896,0.107962,0.041005,0.107962,MnCr15 Co2Re Mn2Re CoRe,True,# generated using pymatgen\ndata_MnCrCoRe\n_sy...,Mn Cr Co Re 0 2 o o - 1 2 o o o 1 3 o o o,Mn Cr Co Re 0 2 o o - 1 2 o o o 1 3 o o o i u ...
5068741,61863,agm015934710,AB3C4_37_spg47,m3gnet/alignn_009/In/In4Pr3Sm/xxx_02a-00_agm01...,Pr3SmIn4,"['Pr', 'Sm', 'In']",47,8,"[[2.2537906, 0.0, 0.0], [0.0, 2.4354684, 0.0],...",-33.952294,...,3.754129,-33.952293,0.002074,-0.492613,0.002074,Pr2SmIn3 Pr4SmIn5,NaN,# generated using pymatgen\ndata_Pr3SmIn4\n_sy...,Pr Pr Pr Sm In In In In 0 7 + - o 1 4 + + o 2 ...,Pr Pr Pr Sm In In In In 0 7 + - o 1 4 + + o 2 ...
5068742,61864,agm004966293,ABC2D6_394_spg166,cgat_comp/quaternaries/ABC2D6/runs/batch-000/Z...,Sr2ZnCd6Pd,"['Sr', 'Zn', 'Cd', 'Pd']",166,20,"[[4.949308, 0.0, 0.0], [0.0, 4.949308, 0.0], [...",-35.504737,...,9.916821,-35.504738,0.109197,-0.236618,0.109197,SrCd2Pd ZnPd Sr3Cd11 SrZn3Pd2,True,# generated using pymatgen\ndata_Sr2ZnCd6Pd\n_...,Sr Sr Sr Sr Zn Zn Cd Cd Cd Cd Cd Cd Cd Cd Cd C...,Sr Sr Sr Sr Zn Zn Cd Cd Cd Cd Cd Cd Cd Cd Cd C...


In [5]:
df.columns

Index(['Unnamed: 0', 'mat_id', 'prototype_id', 'location', 'formula',
       'elements', 'spg', 'nsites', 'stress', 'energy_total', 'total_mag',
       'band_gap_ind', 'band_gap_dir', 'dos_ef', 'energy_corrected',
       'e_above_hull', 'e_form', 'e_phase_separation', 'decomposition',
       'in_nomad', 'cif', 'SLICE', 'SLICE PLUS'],
      dtype='object')

In [2]:
import pandas as pd
import glob

def merge_csv_files(input_pattern, output_file):
    """
    Объединяет несколько CSV-файлов в один.

    :param input_pattern: Шаблон для поиска файлов (например, "split_file_part_*.csv").
    :param output_file: Имя выходного файла.
    """
    # Поиск всех файлов по шаблону
    file_list = sorted(glob.glob(input_pattern))
    if not file_list:
        print("Файлы не найдены. Проверьте шаблон.")
        return

    print(f"Найдено {len(file_list)} файлов для объединения.")

    # Чтение и объединение файлов
    combined_df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)

    # Сохранение результата
    combined_df.to_csv(output_file, index=False)
    print(f"Объединённый файл сохранён: {output_file}")

# Параметры
input_pattern = "../data/alexandria/alexandria_split_file_part_*_slice.csv"  # Шаблон для поиска файлов
output_file = "../data/alexandria/alex_full_slice.csv"          # Имя выходного файла

# Запуск функции
merge_csv_files(input_pattern, output_file)

Найдено 10 файлов для объединения.
Объединённый файл сохранён: ../data/alexandria/alex_full_slice.csv


In [6]:
import pandas as pd

def fill_band_gap(df):
    """
    Заполняет столбец 'band_gap' минимальным значением из доступных:
    band_gap_ind, band_gap_dir, dir_gap, ind_gap.
    Если все значения отсутствуют, оставляет NaN.
    """
    # Создаём список столбцов, которые могут содержать ширину запрещённой зоны
    band_gap_columns = ['band_gap_ind', 'band_gap_dir']
    
    # Выбираем минимальное значение из доступных столбцов
    df['band_gap_n'] = df[band_gap_columns].min(axis=1)
    return df


def unify_formation_energy(df):
    """
    Создаёт единый столбец 'formation_energy' из возможных вариантов:
    formation_energy_per_atom, e_form, energy_per_atom.
    Если значения конфликтуют, выбирается первое непустое.
    """
    # Список возможных столбцов для энергии образования
    formation_energy_columns = ['e_form']
    
    # Создаём новый столбец, выбирая первое непустое значение из списка
    df['formation_energy'] = df[formation_energy_columns].bfill(axis=1).iloc[:, 0]
    return df


def count_non_empty_values(df, columns):
    """
    Подсчитывает количество непустых значений в указанных столбцах.
    Возвращает словарь с результатами.
    """
    non_empty_counts = {col: df[col].notna().sum() for col in columns}
    return non_empty_counts



# Предполагаем, что у вас есть DataFrame `df`
# df = pd.read_csv("your_dataset.csv")  # загрузите ваш датасет

# Шаг 1: Заполнить столбец band_gap
df = fill_band_gap(df)

# Шаг 2: Создать единый столбец formation_energy
df = unify_formation_energy(df)

# Шаг 3: Подсчитать количество непустых значений в результирующих столбцах
result_columns = ['band_gap_n', 'formation_energy']
counts = count_non_empty_values(df, result_columns)

# Вывести результаты
print("Количество непустых значений:")
for col, count in counts.items():
    print(f"{col}: {count}")

Количество непустых значений:
band_gap_n: 5068744
formation_energy: 5068744


In [7]:
df = df[['band_gap_n', 'formation_energy', 'SLICE', 'SLICE PLUS']]

In [8]:
df = df.loc[df['band_gap_n'].notna() & df['formation_energy'].notna()]

In [9]:
df.shape

(5068744, 4)

In [85]:
def create_input(row):
    return f"band_gap={row['band_gap_n']} e_form={row['formation_energy']}"

new_data = {
    'input': df.apply(create_input, axis=1),
    'output': df['SLICE']
}

df_for_llama = pd.DataFrame(new_data)
df_for_llama = df_for_llama.dropna()
df_for_llama.to_csv('../data/df_for_llama_bg_eform_slice.csv', index=False)

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Функция для создания входных данных
def create_input(row):
    return f"{row['band_gap_n']} {row['formation_energy']}"

# Создание нового датасета
new_data = {
    'input': df.apply(create_input, axis=1),
    'output': df['SLICE']
}

df_for_llama = pd.DataFrame(new_data)

# Удаление строк с пропущенными значениями
df_for_llama = df_for_llama.dropna().drop_duplicates()

# Разделение на обучающую и валидационную части
train_df, valid_df = train_test_split(df_for_llama, test_size=0.1, random_state=42)

# Сохранение в CSV файлы
train_df.to_csv('../data/train_alex_full_gpt2.csv', index=False, header=True)
valid_df.to_csv('../data/validation_alex_full_gpt2.csv', index=False, header=True)

print("Датасет успешно создан и разделен на train и validation части!")

Датасет успешно создан и разделен на train и validation части!


In [17]:
train_df

,input,output
1226941,0.0 0.093049474,Ac Ac Si Os 0 1 o o + 0 1 - o o 0 2 o o o 0 3 ...
2553221,0.0 -0.48427057,Pr Pt Pt Pt Pt Pt 0 3 o o o 0 1 o o o 1 2 o o ...
4877144,0.0 1.8508282,Al C C 0 2 o - - 0 1 - o o 1 2 o - o 1 2 o o -
891844,0.0 -0.56779855,K U Cu Cu Se Se Se Se 0 4 o - - 0 5 o o - 1 2 ...
1860706,0.0 -0.7620643,Ce Ce Ce Ce Ce Ce Tm Tm Sb Sb Sb Sb Sb Sb 0 11...
...,...,...
1573788,0.0 -2.574238,Sr Sr Sr Ca Mn Mn Cr O O O O O O O O O O 0 1 o...
2239845,0.0 0.31608596,Ba Ba As Ru Ru 0 2 + + o 0 1 o + - 0 3 o o - 0...
4938227,0.0 0.07110555,Th Th Sc Zn 0 1 - o + 0 1 o o o 0 3 o o o 0 2 ...
4314800,0.0 -0.29178774,Er Er Er Er Mg Hg Hg Hg Hg Hg Hg Hg 0 9 o o o ...


## Fine-tuning llama

In [1]:
from huggingface_hub import login
login()

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "C:\Programs\crystall_generation\Llama-3.2-3B-Instruct\original"
tokenizer_path = "C:\Programs\crystall_generation\Llama-3.2-3B-Instruct\original"

# Загрузка модели
model = AutoModelForCausalLM.from_pretrained(model_path)

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory C:\Programs\crystall_generation\Llama-3.2-3B-Instruct\original.

In [ ]:
import pandas as pd
from datasets import Dataset

# Загрузка данных из CSV
data = pd.read_csv('path/to/your/dataset.csv')

# Преобразование в Dataset
dataset = Dataset.from_pandas(data)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    inputs = examples['input']
    targets = examples['output']
    
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(targets, max_length=512, truncation=True).input_ids
    
    # Установка -100 для padding tokens в labels
    for i in range(len(labels)):
        label = labels[i]
        if len(label) < 512:
            label += [-100] * (512 - len(label))
        labels[i] = label
    
    model_inputs["labels"] = labels
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

d:\Projects\projects_python\crystall_generation\.venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_11700\3178421298.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,  # rank of the low-rank decomposition
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

NameError: name 'model' is not defined

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
model.save_pretrained("path/to/save/fine-tuned-model")
tokenizer.save_pretrained("path/to/save/fine-tuned-model")

In [1]:
# Генерация текста
input_text = "band_gap=0.0 spacegroup.number=139"
from transformers import pipeline

nlp = pipeline("text-generation", model="C:\Programs\crystall_generation\models\llama")

result = nlp("input_text")
print(result)

C:\Users\igorg\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:31<00:00, 15.50s/it]
Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.66 GiB is allocated by PyTorch, and 1.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Valid fine-tuning models

for valid models we synthesised a small dataset with inputs using description of banging_gap & spacegroup.numbers. We assume that generated CIF files will be converted to pymatgen.Structure without any problems. So metric quality of model will be percent of converted structures.

In [9]:
df.describe()

,Unnamed: 0,formation_energy_per_atom,band_gap,e_above_hull,spacegroup.number
count,45229.000000,45229.000000,45229.000000,45229.000000,45229.000000
mean,22614.183710,-1.214873,0.789139,0.017189,123.126843
std,13056.910572,1.027732,1.417502,0.023133,79.655043
min,0.000000,-5.153569,0.000000,0.000000,1.000000
25%,11307.000000,-2.001845,0.000000,0.000000,57.000000
50%,22614.000000,-0.806754,0.000000,0.003469,139.000000
75%,33921.000000,-0.403862,1.133400,0.030189,194.000000
max,45230.000000,0.079825,17.902300,0.079999,229.000000


In [10]:
import numpy as np
import pandas as pd

# Статистические характеристики из датасета
stats = {
    'band_gap': {'mean': 0.789139, 'std': 1.417502, 'min': 0.0, 'max': 17.902300},
    'spacegroup.number': {'min': 1, 'max': 229}
}

# Генерация 100 случайных значений
np.random.seed(42)  # Фиксируем seed для воспроизводимости

# Генерация значений для band_gap (нормальное распределение)
band_gap = np.clip(
    np.random.normal(stats['band_gap']['mean'], stats['band_gap']['std'], 100),
    stats['band_gap']['min'],
    stats['band_gap']['max']
)

# Генерация значений для spacegroup.number (равномерное распределение)
spacegroup_number = np.random.randint(stats['spacegroup.number']['min'], stats['spacegroup.number']['max'] + 1, 100)

# Формирование строк input
inputs = [
    f"band_gap={bg:.4f} spacegroup.number={sg}"
    for bg, sg in zip(band_gap, spacegroup_number)
]

# Создание DataFrame для сохранения результатов
synthetic_data = pd.DataFrame({'input': inputs})

# Сохранение в CSV файл
synthetic_data.to_csv('../data/banchmark/synthetic_inputs.csv', index=False)


In [12]:
synthetic_data.head(2)

,input
0,band_gap=1.4932 spacegroup.number=137
1,band_gap=0.5931 spacegroup.number=62


let's take 100 samples from mp_20 dataset for comparison between synthesed and real data for model

In [11]:
real_sample = df_for_llama.sample(n=100)
remaining_data = df_for_llama.drop(real_sample.index)

real_sample[['input']].to_csv('../data/banchmark/real_inputs.csv', index=False)

# Сохранение оставшихся данных в CSV файл
remaining_data.to_csv('../data/df_for_llama_m100.csv', index=False)

In [12]:
remaining_data.head(2)

,input,output
0,band_gap=0.2132999999999998 spacegroup.number=8,data_Na3MnCoNiO6\n_symmetry_space_group_name_H...
1,band_gap=0.0 spacegroup.number=139,data_Nd(Al2Cu)4\n_symmetry_space_group_name_H-...


In [ ]:
import pandas as pd
from pymatgen.core.structure import Structure
from pymatgen.io.cif import CifParser
from transformers import pipeline

def validate_cif(real_csv, synthetic_csv, model_path):
    # Загрузка модели
    generator = pipeline("text-generation", model=model_path)

    def process_dataset(csv_file, dataset_type):
        # Загрузка CSV файла
        df = pd.read_csv(csv_file)
        
        results = []
        successful_count = 0

        for idx, row in df.iterrows():
            input_text = row['input']
            
            # Генерация CIF строки с помощью модели
            generated_cif = generator(input_text, max_length=500)[0]['generated_text']
            
            try:
                # Попытка преобразовать CIF в структуру pymatgen
                structure = Structure.from_str(generated_cif, fmt="cif")
                conclusion = "Successful"
                successful_count += 1
            except Exception as e:
                # Если возникла ошибка, считаем генерацию неудачной
                conclusion = "Failed"

            # Добавляем результат в список
            results.append({
                'input': input_text,
                'generated_cif': generated_cif,
                'conclusion': conclusion
            })

        # Создание DataFrame с результатами
        result_df = pd.DataFrame(results)
        
        # Сохранение результатов в CSV файл
        result_df.to_csv(f'{dataset_type}_validation_results.csv', index=False)
        
        # Подсчет процента успешных генераций
        success_rate = (successful_count / len(df)) * 100
        
        return result_df, success_rate

    # Обработка реальных данных
    real_results, real_success_rate = process_dataset(real_csv, 'real')

    # Обработка синтезированных данных
    synthetic_results, synthetic_success_rate = process_dataset(synthetic_csv, 'synthetic')

    # Вывод результатов
    print(f"Real data success rate: {real_success_rate:.2f}%")
    print(f"Synthetic data success rate: {synthetic_success_rate:.2f}%")

    return real_results, synthetic_results

# Пример использования функции
real_csv = 'real_inputs_100.csv'
synthetic_csv = 'synthetic_inputs.csv'
model_path = './fine_tuned_llama'  # Укажите путь к вашей fine-tuned модели

real_results, synthetic_results = validate_cif(real_csv, synthetic_csv, model_path)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pymatgen.core.structure import Structure
from transformers import pipeline

def analyze_attempts(real_csv, synthetic_csv, model_path, attempts_list):
    # Загрузка модели
    generator = pipeline("text-generation", model=model_path)

    def process_dataset(csv_file, dataset_type):
        # Загрузка CSV файла
        df = pd.read_csv(csv_file)
        
        success_rates = []  # Для хранения процентов успешных генераций
        
        for attempts in attempts_list:
            successful_count = 0

            for idx, row in df.iterrows():
                input_text = row['input']
                success = False  # Флаг успеха для текущего input

                for _ in range(attempts):
                    # Генерация CIF строки с помощью модели
                    generated_cif = generator(input_text, max_length=500)[0]['generated_text']

                    try:
                        # Попытка преобразовать CIF в структуру pymatgen
                        structure = Structure.from_str(generated_cif, fmt="cif")
                        success = True  # Если успешно, выходим из цикла
                        break
                    except Exception:
                        continue  # Если неудачно, пробуем снова

                if success:
                    successful_count += 1

            # Подсчет процента успешных генераций
            success_rate = (successful_count / len(df)) * 100
            success_rates.append(success_rate)

        return success_rates

    # Обработка реальных данных
    real_success_rates = process_dataset(real_csv, 'real')

    # Обработка синтезированных данных
    synthetic_success_rates = process_dataset(synthetic_csv, 'synthetic')

    # Построение графика
    plt.figure(figsize=(10, 6))
    plt.plot(attempts_list, real_success_rates, label='Real Data', marker='o')
    plt.plot(attempts_list, synthetic_success_rates, label='Synthetic Data', marker='x')
    
    plt.title('Success Rate vs Number of Attempts')
    plt.xlabel('Number of Attempts per Input')
    plt.ylabel('Success Rate (%)')
    plt.legend()
    plt.grid(True)
    
    # Сохранение графика в файл
    plt.savefig('success_rate_vs_attempts.png')  # Сохраняем график в PNG файл
    plt.show()

    # Вывод данных в текстовом формате
    with open('success_rates.txt', 'w') as f:
        f.write("Number of Attempts: {}\n".format(attempts_list))
        f.write("Real Data Success Rates (%): {}\n".format(real_success_rates))
        f.write("Synthetic Data Success Rates (%): {}\n".format(synthetic_success_rates))

    return real_success_rates, synthetic_success_rates

# Пример использования функции
real_csv = 'real_inputs_100.csv'
synthetic_csv = 'synthetic_inputs.csv'
model_path = './fine_tuned_llama'  # Укажите путь к вашей fine-tuned модели
attempts_list = [1, 2, 3, 5, 10]  # Список количества попыток

real_success_rates, synthetic_success_rates = analyze_attempts(real_csv, synthetic_csv, model_path, attempts_list)